In [3]:
# imports
import os
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [6]:
openai = OpenAI()

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the \
the dashboard in a website and provides a short executive summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nPlease provide a detailed summary of the report for the year in markdown for its user (CFO); \
The summary should be in a suitable form which could be sent through a mail for the exective.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [10]:
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [11]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [12]:
display_summary("https://community.fabric.microsoft.com/t5/Data-Stories-Gallery/Revenue-amp-Profitability-Dashboard/td-p/4780272")

# Executive Summary: Revenue & Profitability Dashboard

**To:** [CFO Name]  
**From:** [Your Name]  
**Date:** [Current Date]  
**Subject:** Yearly Analysis of Revenue & Profitability  

---

Dear [CFO Name],

I am pleased to present the Year-over-Year analysis derived from the Revenue & Profitability Dashboard. This dashboard has been designed to provide concise insights into our core financial performance metrics, enabling data-driven decision-making at the executive level.

### Key Metrics Overview:
- **Revenue**: Comprehensive insights into total revenue across various regions and product categories, indicating sustainable growth patterns.
- **Profit**: Detailed profitability analysis segmented by customer groups, revealing key opportunities for margin improvement and cost optimization.
- **Unit Sales**: Analysis of unit sales trends that highlight product performance and demand fluctuations.

### Insights by Segment:
- **Regional Performance**: Comparative analysis of revenue and profitability by region helps identify areas of growth and those requiring strategic intervention.
- **Product Performance**: A focused review of individual product lines shows which offerings are driving profitability and where we might consider realignment or innovation.

### Dashboard Features:
- A **clean and focused layout** reduces cognitive load, allowing for quick assimilation and understanding of critical data points.
- **Contextual metrics** that align with our overarching business strategy, ensuring that our analysis supports our organizational goals.
- **Clear comparison points** are established to aid executives in making informed and timely decisions.
- Insightful details are presented at both product and regional levels, facilitating targeted strategies for improvement.

### Conclusion:
The integration of design and context in our dashboard framework turns our data into strategic tools, empowering us to make faster and more informed decisions that drive real business impact.

Please feel free to reach out for a more detailed discussion or specific metrics that may interest you.

Best regards,

[Your Name]  
[Your Position]

--- 

*Note: For additional inquiries or insights, feel free to follow our updates on LinkedIn or contact me directly.*